In [15]:
import pandas as pd
from collections import defaultdict
from tqdm import tqdm
import pickle
import multitasking
import os
import signal
import math
import numpy as np
# from utils import evaluate
from random import shuffle
import warnings
import math

warnings.filterwarnings('ignore')

multitasking.set_max_threads(10)
multitasking.set_engine('process')
signal.signal(signal.SIGINT, multitasking.killall)

<function _signal.default_int_handler>

In [16]:
df_query = pd.read_pickle('data/query.pkl')
df_click = pd.read_pickle('data/click.pkl')

In [17]:
df_click.head()

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,0,30760,1508211672520,4,1,17,1,25,2
1,0,157507,1508211702520,4,1,17,1,25,2
2,1,289197,1508211316889,4,1,17,1,25,6
3,1,63746,1508211346889,4,1,17,1,25,6
4,2,36162,1508211438695,4,3,20,1,25,2


In [19]:
 user_item_ = df_click.groupby('user_id')['click_article_id'].agg(
        lambda x: list(x)).reset_index()

In [21]:
    user_item_dict = dict(
        zip(user_item_['user_id'], user_item_['click_article_id']))

In [20]:
 user_item_.head()

,user_id,click_article_id
0,0,"[30760, 157507]"
1,1,"[289197, 63746]"
2,2,"[36162, 168401]"
3,3,"[50644, 36162]"
4,4,"[42567, 39894]"


In [22]:
user_item_dict[0]

[30760, 157507]

In [4]:
def cal_item_sim(df):
    user_item_ = df.groupby('user_id')['click_article_id'].agg(
        lambda x: list(x)).reset_index()
    user_item_dict = dict(
        zip(user_item_['user_id'], user_item_['click_article_id']))

    item_cnt = defaultdict(int)
    sim_dict = {}

    for user, items in tqdm(user_item_dict.items()):
        for item in items:
            item_cnt[item] += 1
            sim_dict.setdefault(item, {})

            for relate_item in items:
                if item == relate_item:
                    continue

                sim_dict[item].setdefault(relate_item, 0)

                sim_dict[item][relate_item] += 1 / math.log(1 + len(items))

    sim_dict_corr = sim_dict.copy()
    for item, relate_items in tqdm(sim_dict.items()):
        for relate_item, cij in relate_items.items():
            sim_dict_corr[item][relate_item] = cij / \
                math.sqrt(item_cnt[item] * item_cnt[relate_item])

    return sim_dict_corr, user_item_dict

In [5]:
# user_item_dict[196381]

NameError: name 'user_item_dict' is not defined

In [24]:
user_item_dict


{0: [30760, 157507],
 1: [289197, 63746],
 2: [36162, 168401],
 3: [50644, 36162],
 4: [42567, 39894],
 5: [211442, 234481],
 6: [62464, 10023],
 7: [50644, 211442],
 8: [70986, 50644],
 9: [70986, 211442, 211455],
 10: [50644, 159195],
 11: [50644, 234481],
 12: [211442, 211455],
 13: [36162, 277107],
 14: [70986, 36162],
 15: [277107, 342473, 206415],
 16: [50644, 211442],
 17: [156279, 158331, 363916],
 18: [70986, 224730],
 19: [70986, 205824],
 20: [285433, 285300],
 21: [205958, 70758],
 22: [107014, 107190],
 23: [309535, 309311],
 24: [211442, 156279],
 25: [70240, 74665],
 26: [156279, 36162, 211722],
 27: [36162, 50644],
 28: [284470, 224730],
 29: [202355, 209122],
 30: [70986, 209122],
 31: [156279, 161526],
 32: [50644, 211442, 156279],
 33: [331116, 36162],
 34: [79851, 158158, 207603, 108903],
 35: [205824, 50644],
 36: [205824, 277107],
 37: [36162, 87694],
 38: [156279, 158331, 234481],
 39: [255035, 255683],
 40: [237614, 243234, 233578],
 41: [70986, 70758, 50644, 20

In [23]:
item_sim, user_item_dict = cal_item_sim(df_click)

100%|██████████████████████████████████████████████████████████████████████████| 35362/35362 [00:04<00:00, 8222.10it/s]


In [14]:
item_sim[233917]

{182513: 0.04431851472321662,
 156624: 0.01983108836635084,
 71500: 0.006955696389489084,
 50823: 0.016281006232645702,
 166430: 0.00686133604281548,
 236961: 0.04401027971023317,
 235715: 0.0021667942665138436,
 118464: 0.017878786934717648,
 161409: 0.0212494899477255,
 348111: 0.036921334035644356,
 162655: 0.008457799916423247,
 33333: 0.00451737467667151,
 284547: 0.0060391788595561095,
 214691: 0.024540741004317176,
 214690: 0.030056146685186736,
 272143: 0.016141030958315438,
 199198: 0.010102360252528884,
 224658: 0.004104346924096312,
 198659: 0.009024815676666308,
 336476: 0.06662145609475435,
 29945: 0.016171971264318213,
 158229: 0.00846957257794673,
 284474: 0.022976562957490544,
 64329: 0.00910399528357524,
 236726: 0.024076478685028036,
 234000: 0.04121030389202205,
 288841: 0.0060722586106951755,
 75302: 0.032886984057221645,
 75865: 0.018987309098271846,
 100838: 0.0232546094396353,
 129029: 0.00794281219172258,
 175040: 0.009763559684346716,
 353713: 0.009493654549135

In [27]:
df_query.head()

,user_id,click_article_id
0,196387,158536
1,196397,160417
2,196401,162765
3,196405,324823
4,196420,156447


In [28]:
user_item_dict[196387]

[156624]

In [30]:
interacted_items = user_item_dict[196387]

In [31]:
interacted_items[::-1]

[156624]

In [32]:
sorted(item_sim[156624].items(), key=lambda d: d[1], reverse=True)[0:100]

[(156560, 0.07939810685477457),
 (160417, 0.053962613625577756),
 (272143, 0.05352856277561936),
 (158229, 0.05129699899794728),
 (337890, 0.050472777911251085),
 (64329, 0.04657100993660928),
 (158536, 0.04466605860405093),
 (156447, 0.04369797690904955),
 (157332, 0.042521491645693354),
 (336476, 0.04180522070392955),
 (162655, 0.04131553555422808),
 (50823, 0.039667121571765586),
 (162765, 0.038900901721944235),
 (160974, 0.038045287589420725),
 (159762, 0.03690500130324161),
 (160132, 0.036110262417199675),
 (182513, 0.034176922934445596),
 (156619, 0.03348286161034781),
 (199197, 0.033312704655656375),
 (235230, 0.0327327414497434),
 (348111, 0.031870820870642295),
 (324823, 0.030895555471963466),
 (233717, 0.030515150140746756),
 (284474, 0.030192438201347708),
 (284547, 0.0291688858410247),
 (123909, 0.029134049885786648),
 (336221, 0.02908886447654599),
 (166338, 0.028155970655249815),
 (160148, 0.026383683330984748),
 (199198, 0.026087192057635676),
 (293301, 0.026032245861790

In [7]:
def recall(df_query, item_sim, user_item_dict):
    data_list = []

    for user_id, item_id in tqdm(df_query.values):
        rank = defaultdict(int)
        
        if user_id not in user_item_dict:
            continue
        
        interacted_items = user_item_dict[user_id]


        for loc, item in enumerate(interacted_items):
            for relate_item, wij in sorted(item_sim[item].items(), key=lambda d: d[1], reverse=True)[0:100]:
                if relate_item not in interacted_items:
                    rank.setdefault(relate_item, 0)
                    rank[relate_item] += wij

        sim_items = sorted(
            rank.items(), key=lambda d: d[1],  reverse=True)[:50]
        item_ids = [item[0] for item in sim_items]
        item_sim_scores = [item[1] for item in sim_items]

        df_temp = pd.DataFrame()
        df_temp['article_id'] = item_ids
        df_temp['sim_score'] = item_sim_scores
        df_temp['user_id'] = user_id

        if item_id == -1:
            df_temp['label'] = np.nan
        else:
            df_temp['label'] = 0
            df_temp.loc[df_temp['article_id'] == item_id, 'label'] = 1

        df_temp = df_temp[[
            'user_id',  'article_id', 'sim_score', 'label'
        ]]
        df_temp['user_id'] = df_temp['user_id'].astype('int')
        df_temp['article_id'] = df_temp['article_id'].astype('int')

        data_list.append(df_temp)

    df_data = pd.concat(data_list, sort=False)
    return df_data

In [8]:
df_data = recall(df_query, item_sim, user_item_dict)

100%|████████████████████████████████████████████████████████████████████████████| 50887/50887 [13:17<00:00, 63.83it/s]


In [9]:
df_data.head()

,user_id,article_id,sim_score,label
0,196387,156560,0.079398,0.0
1,196387,160417,0.053963,0.0
2,196387,272143,0.053529,0.0
3,196387,158229,0.051297,0.0
4,196387,337890,0.050473,0.0


In [25]:
# 必须加，对其进行排序
df_data = df_data.sort_values(['user_id', 'sim_score'], ascending=[
                              True, False]).reset_index(drop=True)

In [27]:
df_data.head(10)

,user_id,article_id,sim_score,label
0,196387,156560,0.079398,0.0
1,196387,160417,0.053963,0.0
2,196387,272143,0.053529,0.0
3,196387,158229,0.051297,0.0
4,196387,337890,0.050473,0.0
5,196387,64329,0.046571,0.0
6,196387,158536,0.044666,1.0
7,196387,156447,0.043698,0.0
8,196387,157332,0.042521,0.0
9,196387,336476,0.041805,0.0


In [28]:
from random import sample
prediction = df_data[df_data['label'].isnull()]
prediction.sort_values(['user_id', 'sim_score'],
                       inplace=True,
                       ascending=[True, False])

all_articles = set(df_click['click_article_id'].values)

lines = []
test_users = df_query[df_query['click_article_id'] == -1]['user_id'].unique()

for test_user in tqdm(test_users):
    g = prediction[prediction['user_id'] == test_user]
    g = g.head(5)
    items = g['article_id'].values.tolist()
    
    if len(set(items)) < 5:
        buchong = all_articles - set(items)
        buchong = sample(buchong, 5 - len(set(items)))
        items += buchong
    
    assert len(set(items)) == 5

    lines.append([test_user] + items)

df_sub = pd.DataFrame(lines)
df_sub.columns = ['user_id', 'article_1', 'article_2',
                  'article_3', 'article_4', 'article_5']
df_sub.sort_values(['user_id'], inplace=True)
os.makedirs('sub', exist_ok=True)
df_sub.to_csv('sub/itemcf.csv', index=False)

100%|███████████████████████████████████████████████████████████████████████████| 50000/50000 [03:14<00:00, 257.15it/s]


In [13]:
# df_query = pd.read_pickle('data/query.pkl')
# total = df_query[df_query['click_article_id'] != -1].user_id.nunique()

# hitrate_5, mrr_5, hitrate_10, mrr_10, hitrate_20, mrr_20, hitrate_40, mrr_40, hitrate_50, mrr_50 = evaluate(df_data[df_data['label'].notnull()], total)
# hitrate_5, mrr_5, hitrate_10, mrr_10, hitrate_20, mrr_20, hitrate_40, mrr_40, hitrate_50, mrr_50